# Part-4: Transfer Learning with Progressive data and TFDS (tensorflow data sets)

# Also called as progressive training

### STEP 1: Tensorflow settings for XLA, warning, memory allocation

In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"  # must run before `import tensorflow`

# (Optional) also reduce logs:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

In [2]:
import tensorflow as tf
tf.config.optimizer.set_jit("autoclustering")
print("JIT enabled? ", getattr(tf.config.optimizer, "get_jit", lambda: "n/a")())

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

JIT enabled?  autoclustering


### STEP 2:  Import libraries

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd

import tensorflow_hub as hub

import helper_functions_updated as hf

/home/raghu/tf-env/lib/python3.12/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


### STEP 3:  Import dataset and check files

            ○ its easy to split data
            ○ 2-5x Faster Training ⚡
            ○ Optimal for TensorFlow's data pipeline
            ○ Better shuffling & prefetching - Excellent
            ○ Single-file simplicity
            ○ Built-in compression
            ○ Memory usage - Lower

In [4]:
import tensorflow_datasets as tfds


(ds_train, ds_val), ds_info = tfds.load(
                                        "food101",
                                        split=["train", "validation"],
                                        shuffle_files=True,
                                        as_supervised=True,
                                        with_info=True,
                                    )

print(ds_info.features)
print(ds_info.splits)
print(ds_info.supervised_keys)
print(ds_info.description)

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=101),
})
{'train': <SplitInfo num_examples=75750, num_shards=32>, 'validation': <SplitInfo num_examples=25250, num_shards=16>}
('image', 'label')
This dataset consists of 101 food categories, with 101'000 images. For each class, 250 manually reviewed test images are provided as well as 750 training images. On purpose, the training images were not cleaned, and thus still contain some amount of noise. This comes mostly in the form of intense colors and sometimes wrong labels. All images were rescaled to have a maximum side length of 512 pixels.


I0000 00:00:1767897900.279511   30678 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5518 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:64:00.0, compute capability: 8.9


### STEP 4:  Data Split

In [5]:
import tensorflow_datasets as tfds
     
# Progressive training splits
ds_10p = tfds.load(
                        "food101",
                        split="train[:10%]",
                        as_supervised=True,
                    )

ds_50p = tfds.load(
                        "food101",
                        split="train[:50%]",
                        as_supervised=True,
                    )

ds_100p = tfds.load(
                        "food101",
                        split="train",
                        as_supervised=True,
                    )

# Fixed validation set
ds_validation = tfds.load(
                        "food101",
                        split="validation",
                        as_supervised=True,
                    )

In [6]:
def count_examples(ds):
    return sum(1 for _ in ds)

print("Train 10%:", count_examples(ds_10p))
print("Train 50%:", count_examples(ds_50p))
print("Train 100%:", count_examples(ds_100p))
print("Validation:", count_examples(ds_validation))

Train 10%: 7575
Train 50%: 37875
Train 100%: 75750
Validation: 25250


### STEP 5:  Set Variables

In [7]:
BATCH_SIZE = 32
IMG_SIZE = 384
BUFFER_SIZE = 1000   # good default for Food101
EPOCHS = 5

### STEP 6:  Image processing + Augmentation, shuffle and prefetch for training dataset, which should be outside model

      RAW IMAGE
         ↓
      PREPROCESSING  (deterministic, always on)
         ↓
      AUGMENTATION   (random, TRAIN ONLY)
         ↓
      MODEL

In [8]:
# Method defined here is new and moved to tf.keras.layers from tf.keras.layers.experimental.preprocessing
# Data Augmentation for incresing the diversity of the training data
# Consider XLA (Accelerated Linear Algebra) for speeding up the training which do not support some layers
data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal"),
                                         tf.keras.layers.RandomRotation(0.2),
                                         tf.keras.layers.RandomZoom(height_factor=0.2, width_factor=0.2)],                                      
                                         # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNetV2B0
                                        name ="data_augmentation")

In [9]:
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

# Preprocessing image
def preprocess_fn(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)    
    return image, label


def build_training_dataset(dataset, BUFFER_SIZE, BATCH_SIZE):    
    # shuffle images before batching up
    dataset_processed = (
                            dataset
                            .shuffle(BUFFER_SIZE)
                            .map(preprocess_fn, num_parallel_calls=tf.data.AUTOTUNE)
                            .map(lambda x, y: (data_augmentation(x, training=True), y))
                            .batch(BATCH_SIZE)
                            .prefetch(1)
                        )
    return dataset_processed

def build_eval_dataset(dataset, BATCH_SIZE):
    return (
        dataset
        .map(preprocess_fn, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(BATCH_SIZE)
        .prefetch(1)
    )

In [10]:
ds_train_10 = build_training_dataset(ds_10p, BUFFER_SIZE, BATCH_SIZE)
ds_train_50 = build_training_dataset(ds_50p, BUFFER_SIZE, BATCH_SIZE)
ds_train_100 = build_training_dataset(ds_100p, BUFFER_SIZE, BATCH_SIZE)
ds_val = build_eval_dataset(ds_validation, BATCH_SIZE)

dataset = [ds_train_10, ds_train_50, ds_train_100, ds_val]
dataset_names = ["ds_train_10", "ds_train_50", "ds_train_100", "ds_val"]
for set, name in zip(dataset, dataset_names):
    for x, y in set.take(1):
        print(f"{name}: shape = {x.shape}, type = {x.dtype}")

I0000 00:00:1767897915.214812   31212 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


ds_train_10: shape = (32, 384, 384, 3), type = <dtype: 'float32'>
ds_train_50: shape = (32, 384, 384, 3), type = <dtype: 'float32'>
ds_train_100: shape = (32, 384, 384, 3), type = <dtype: 'float32'>
ds_val: shape = (32, 384, 384, 3), type = <dtype: 'float32'>


### STEP 7: import Model efficientnetV2S, compile and fit

### Phase 0 setup once

In [11]:
# Creating the checkpoints for model
ckpt_model_10p = tf.keras.callbacks.ModelCheckpoint(filepath="checkpoints/ckpt_model_10p.ckpt",
                                               save_weights_only=True,
                                               save_best_only=True,
                                               monitor="val_accuracy",
                                               save_freq="epoch",
                                               verbose=1)                                     

Use the official preprocessing for EfficientNetV2 after augmentation.
- from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

Specify pretrained weights.
- base_model = tf.keras.applications.EfficientNetV2S(include_top=False, weights="imagenet")

Keep BN stable with training=False (what you did).
- a1 = base_model(ax, training=False)

(Optional) enable mixed precision on your 4070 for speed.
- tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [12]:
# (Optional) speedup on RTX 4070
tf.keras.mixed_precision.set_global_policy("mixed_float16")

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


In [13]:
def build_food101_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=101, dropout_rate=0.5):

    # Setup base model, freezing the base model layers and considering pretrained weights on imagenet
    base_model = tf.keras.applications.EfficientNetV2S(include_top=False, 
                                                       weights="imagenet", 
                                                       input_shape=input_shape
                                                       )  
    
    inputs = tf.keras.layers.Input(shape=input_shape, name="input_layer")
 
    a1 = base_model(inputs, training=False)   # training = True, only last 20 layers are trainable,  
                                          # training = False, BatchNorms and Dropouts are freezed during training  very important

    # Hybrid pooling
    gap = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(a1)
    gmp = tf.keras.layers.GlobalMaxPooling2D(name="max_pooling_layer")(a1)
    a2 = tf.keras.layers.Concatenate(name="max_avg_pooling")([gap, gmp])

    # Dropout
    a3 = tf.keras.layers.Dropout(dropout_rate)(a2)  # add dropout layer and a bit stronger on small data
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax", dtype="float32", name="output_layer")(a3)

    return tf.keras.Model(inputs, outputs), base_model


In [14]:
model, base_model = build_food101_model(
                                            input_shape=(384, 384, 3),
                                            num_classes=101,
                                            dropout_rate=0.5
                                        )

### Phase 1 10% data (Feature extraction)

In [15]:
# Freeze backbone (Phase 1)
for layer in base_model.layers:
    layer.trainable = False

# 
model_10p = model

# Compile (feature extraction)    
model_10p.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(), # add label smoothing
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  metrics=["accuracy"]
                  )

# Train
history_10p = model_10p.fit(ds_train_10,
                          epochs=5,
                          validation_data=ds_val,                          
                          callbacks=[hf.create_tensorboard_callback("transfer_learning", "10_percent_data_aug"), 
                                      ckpt_model_10p]
                        )

Saving TensorBoard log files to: /mnt/d/04_Food101-EfficientNetV2S-model/notebook/transfer_learning/10_percent_data_aug/20260108-134520
Epoch 1/5


2026-01-08 13:45:25.344710: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


237/237 [==============================] - ETA: 0s - loss: 6.9153 - accuracy: 0.1048
Epoch 1: val_accuracy improved from -inf to 0.36491, saving model to checkpoints/ckpt_model_10p.ckpt
237/237 [==============================] - 176s 633ms/step - loss: 6.9153 - accuracy: 0.1048 - val_loss: 3.0533 - val_accuracy: 0.3649
Epoch 2/5
237/237 [==============================] - ETA: 0s - loss: 4.7866 - accuracy: 0.2453
Epoch 2: val_accuracy improved from 0.36491 to 0.44214, saving model to checkpoints/ckpt_model_10p.ckpt
237/237 [==============================] - 164s 690ms/step - loss: 4.7866 - accuracy: 0.2453 - val_loss: 2.7120 - val_accuracy: 0.4421
Epoch 3/5
237/237 [==============================] - ETA: 0s - loss: 4.0621 - accuracy: 0.3158
Epoch 3: val_accuracy improved from 0.44214 to 0.44657, saving model to checkpoints/ckpt_model_10p.ckpt
237/237 [==============================] - 130s 547ms/step - loss: 4.0621 - accuracy: 0.3158 - val_loss: 2.6758 - val_accuracy: 0.4466
Epoch 4/5
2

Final validation Top-1 accuracy: 53.56%

This is exactly where a frozen-backbone EfficientNetV2S should land with:

            only 10% of Food101
            strong augmentation
            no label smoothing
            mixed precision
            short training (5 epochs)
            You nailed it.

🧠 How to correctly interpret this (important)

        1️⃣ Val accuracy > train accuracy → EXPECTED
            This is not a bug.
        Why:
            Training data has augmentation + dropout
            Validation data is clean
            Backbone is frozen
            Food101 training split is noisy by design
            This is normal and healthy in Phase-1.

        2️⃣ Loss curve behavior is correct

            Train loss steadily ↓ (6.86 → 3.50)
            Val loss steadily ↓ (3.10 → 2.11)
            No NaNs
            No instability

        This confirms:
            Mixed precision is stable
            Output dtype fix worked

Learning rate is correct

### Phase 2 - 50% data 

- Epochs: 5 → 20
- Trainable: unfreeze last ~20–30 layers only 
- LR: 3e-5 (big drop from Phase 1), AdamW weight_decay=1e-5.

Why freezing BatchNorm is so important (Food101 specific)

EfficientNet’s BatchNorm layers:
    Were trained on ImageNet
    Contain very strong, stable statistics

    Are extremely sensitive to:
        Small batch sizes
        Dataset shift
        Aggressive augmentation

If you unfreeze BN layers:

        Validation accuracy often drops
        Training becomes unstable
        Convergence slows or oscillates

In [16]:
base_model.trainable = True

# Unfreeze last N layers of the backbone
N = 30

# 1) freeze everything first
for layer in base_model.layers:
    layer.trainable = False

# 2) unfreeze the tail, but keep BN frozen
for layer in base_model.layers[-N:]:
    layer.trainable = not isinstance(layer, tf.keras.layers.BatchNormalization)

In [17]:
ckpt_model_50p = tf.keras.callbacks.ModelCheckpoint(filepath="checkpoints/ckpt_model_50p.ckpt",
                                                    save_weights_only=True,
                                                    save_best_only=True,
                                                    monitor="val_accuracy",
                                                    save_freq="epoch",
                                                    verbose=1)

model_50p = model_10p

# Load best Phase-1 weights
model_50p.load_weights("checkpoints/ckpt_model_10p.ckpt")

# Recompile with lower LR
model_50p.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4, weight_decay=1e-5),
                metrics=["accuracy"])

# Train on 50% data
history_50p = model_50p.fit(ds_train_50,
                            initial_epoch=5,                          
                            epochs=20,                          
                            validation_data=ds_val,                          
                             callbacks=[hf.create_tensorboard_callback("transfer_learning", "50_percent_data_aug"), 
                                       ckpt_model_50p]
                           )

Saving TensorBoard log files to: /mnt/d/04_Food101-EfficientNetV2S-model/notebook/transfer_learning/50_percent_data_aug/20260108-135726
Epoch 6/20
1184/1184 [==============================] - ETA: 0s - loss: 1.8014 - accuracy: 0.5408
Epoch 6: val_accuracy improved from -inf to 0.74008, saving model to checkpoints/ckpt_model_50p.ckpt
1184/1184 [==============================] - 485s 389ms/step - loss: 1.8014 - accuracy: 0.5408 - val_loss: 0.9551 - val_accuracy: 0.7401
Epoch 7/20
1184/1184 [==============================] - ETA: 0s - loss: 1.3068 - accuracy: 0.6530
Epoch 7: val_accuracy improved from 0.74008 to 0.75893, saving model to checkpoints/ckpt_model_50p.ckpt
1184/1184 [==============================] - 467s 394ms/step - loss: 1.3068 - accuracy: 0.6530 - val_loss: 0.8664 - val_accuracy: 0.7589
Epoch 8/20
1184/1184 [==============================] - ETA: 0s - loss: 1.0938 - accuracy: 0.7037
Epoch 8: val_accuracy improved from 0.75893 to 0.78780, saving model to checkpoints/ckpt_mo

### Phase 3 - 100% data 

- Epochs: 40 to 60
- Trainable: unfreeze last ~20–30 layers only
- learning_rate=1e-5, weight_decay=1e-5

In [24]:
base_model.trainable = True

# Unfreeze last N layers of the backbone
N = 50

# 1) freeze everything first
for layer in base_model.layers:
    layer.trainable = False

# 2) unfreeze the tail, but keep BN frozen
for layer in base_model.layers[-N:]:
    layer.trainable = not isinstance(layer, tf.keras.layers.BatchNormalization)

In [25]:
ckpt_model_100p = tf.keras.callbacks.ModelCheckpoint(filepath="checkpoints/ckpt_model_100p.ckpt",
                                                    save_weights_only=True,
                                                    save_best_only=True,
                                                    monitor="val_accuracy",
                                                    save_freq="epoch",
                                                    verbose=1)

model_100p = model_50p

# Load best Phase-2 weights
model_100p.load_weights("checkpoints/ckpt_model_50p.ckpt")

# Recompile with lower LR
model_100p.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, weight_decay=1e-5),
                metrics=["accuracy"])

# Train on 100% data
history_100p = model_100p.fit(ds_train_100,
                              initial_epoch=20,                           
                              epochs=32,                          
                              validation_data=ds_val,                          
                              callbacks=[hf.create_tensorboard_callback("transfer_learning", "100_percent_data_aug"), 
                                         ckpt_model_100p]
                              )

Saving TensorBoard log files to: /mnt/d/04_Food101-EfficientNetV2S-model/notebook/transfer_learning/100_percent_data_aug/20260108-161339
Epoch 21/32
2368/2368 [==============================] - ETA: 0s - loss: 0.7302 - accuracy: 0.8137
Epoch 21: val_accuracy improved from -inf to 0.82622, saving model to checkpoints/ckpt_model_100p.ckpt
2368/2368 [==============================] - 889s 366ms/step - loss: 0.7302 - accuracy: 0.8137 - val_loss: 0.6383 - val_accuracy: 0.8262
Epoch 22/32
2368/2368 [==============================] - ETA: 0s - loss: 0.6780 - accuracy: 0.8259
Epoch 22: val_accuracy improved from 0.82622 to 0.83145, saving model to checkpoints/ckpt_model_100p.ckpt
2368/2368 [==============================] - 870s 367ms/step - loss: 0.6780 - accuracy: 0.8259 - val_loss: 0.6149 - val_accuracy: 0.8314
Epoch 23/32
2368/2368 [==============================] - ETA: 0s - loss: 0.6492 - accuracy: 0.8320
Epoch 23: val_accuracy improved from 0.83145 to 0.83485, saving model to checkpoint

In [1]:
model.save("/mnt/d/04_Food101-EfficientNetV2S-model/models/food101_best.keras", save_format="tf")

NameError: name 'model' is not defined